In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [458 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [897 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,241 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [980 kB]
Hit:13 ht

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
homes_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('homes')

#average four bedroom house sold for each year two decimal places
homes_df.printSchema() #check the schema for non ints or incorrect data types

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# add the number format function to allow nnn.nn for pricing
from pyspark.sql.functions import format_number #normally I would do this at the top of the program
from pyspark.sql.functions import year

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# because I understood this as we are searching for the price of the 4 bedroom home sold in the given year we had to create a new column of just the sold date
four_bd_df = homes_df.withColumn("year", year(homes_df['date']))

four_averages = four_bd_df.groupBy("year").avg()
four_averages.orderBy("year",ascending=False).select("year",format_number("avg(price)",2)).show()


+----+----------------------------+
|year|format_number(avg(price), 2)|
+----+----------------------------+
|2022|                  314,421.54|
|2021|                  313,218.09|
|2020|                  315,949.81|
|2019|                  315,782.18|
+----+----------------------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
three_bd_df = four_bd_df #copy the data frame
three_room_bth = three_bd_df.filter((three_bd_df.bedrooms==3)&(three_bd_df.bathrooms==3)) #filter it down to the 3 bd and 3 bathroom
three_averages = three_room_bth.groupBy("date_built").avg() #group the the 3 bd and 3 baths by build date and average them
three_averages.orderBy("date_built",ascending=False).select("date_built",format_number("avg(price)",2)).show()

+----------+----------------------------+
|date_built|format_number(avg(price), 2)|
+----------+----------------------------+
|      2017|                  292,676.79|
|      2016|                  290,555.07|
|      2015|                  288,770.30|
|      2014|                  290,852.27|
|      2013|                  295,962.27|
|      2012|                  293,683.19|
|      2011|                  291,117.47|
|      2010|                  292,859.62|
+----------+----------------------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
two_fl_three_bd_df = four_bd_df #copy the data frame
two_fl_three_df = two_fl_three_bd_df.filter((two_fl_three_bd_df.bedrooms==3)&(two_fl_three_bd_df.bathrooms==3)&(two_fl_three_bd_df.floors==2)) #filter it down to the 3 bd and 3 bathroom
#add the square footage requirement
two_fl_three_2k_df = two_fl_three_df.filter(two_fl_three_bd_df.sqft_living>=2000)
#two_fl_three_2k_df.show()  # the two_fl_three_2k_df is the filtered down data frame to work with
#get the average price by the year it was built
two_k_averages = two_fl_three_2k_df.groupBy("date_built").avg() #group the the 3 bd and 3 baths by build date and average them
two_k_averages.orderBy("date_built",ascending=False).select("date_built",format_number("avg(price)",2)).show()


+----------+----------------------------+
|date_built|format_number(avg(price), 2)|
+----------+----------------------------+
|      2017|                  280,317.58|
|      2016|                  293,965.10|
|      2015|                  297,609.97|
|      2014|                  298,264.72|
|      2013|                  303,676.79|
|      2012|                  307,539.97|
|      2011|                  276,553.81|
|      2010|                  285,010.22|
+----------+----------------------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

query = '' # I always initialize varibles
query = '''select avg(price),view, date_built, sqft_living
            from homes
                where price >=350000
            GROUP BY price, view, date_built, sqft_living
 '''

start_time = time.time()
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+----------+-----------+
|avg(price)|view|date_built|sqft_living|
+----------+----+----------+-----------+
|  398750.0|  28|      2017|       2258|
|  406369.0|  12|      2012|       2439|
|  421355.0|   9|      2016|       1981|
|  360203.0|   1|      2014|       2273|
|  367581.0|  36|      2010|       2082|
|  377410.0|  40|      2015|       1790|
|  358842.0|  34|      2016|       1900|
|  356872.0|   4|      2016|       2092|
|  403431.0|  43|      2010|       2340|
|  413675.0|  28|      2016|       2064|
|  429295.0|  34|      2015|       1996|
|  797263.0|  72|      2012|       3634|
|  443712.0|   8|      2017|       2207|
|  440746.0|  48|      2013|       1783|
|  717361.0|  74|      2016|       4733|
|  412379.0|  22|      2015|       2430|
|  721804.0|  50|      2017|       4266|
|  446330.0|  35|      2017|       1845|
|  383624.0|   3|      2015|       1635|
|  369169.0|  41|      2011|       2345|
+----------+----+----------+-----------+
only showing top

In [11]:
# 7. Cache the the temporary table home_sales.
#creating a new view of the csv table for different processing.
homes_df.createOrReplaceTempView('home_sales')
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

query = '' # I always initialize varibles
query = '''select avg(price),view, date_built, sqft_living
            from home_sales
                where price >=350000
            GROUP BY price, view, date_built, sqft_living
 '''
#time it
start_time = time.time()

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+----------+-----------+
|avg(price)|view|date_built|sqft_living|
+----------+----+----------+-----------+
|  398750.0|  28|      2017|       2258|
|  406369.0|  12|      2012|       2439|
|  421355.0|   9|      2016|       1981|
|  360203.0|   1|      2014|       2273|
|  367581.0|  36|      2010|       2082|
|  377410.0|  40|      2015|       1790|
|  358842.0|  34|      2016|       1900|
|  356872.0|   4|      2016|       2092|
|  403431.0|  43|      2010|       2340|
|  413675.0|  28|      2016|       2064|
|  429295.0|  34|      2015|       1996|
|  797263.0|  72|      2012|       3634|
|  443712.0|   8|      2017|       2207|
|  440746.0|  48|      2013|       1783|
|  717361.0|  74|      2016|       4733|
|  412379.0|  22|      2015|       2430|
|  721804.0|  50|      2017|       4266|
|  446330.0|  35|      2017|       1845|
|  383624.0|   3|      2015|       1635|
|  369169.0|  41|      2011|       2345|
+----------+----+----------+-----------+
only showing top

In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.parquet('parquet_home_sales', mode='overwrite') #create the parquet object



In [16]:
# 11. Read the parquet formatted data.
p_homes_df=spark.read.parquet('parquet_home_sales')

In [17]:
# 12. Create a temporary table for the parquet data.
p_homes_df.createOrReplaceTempView('parquet_home_sales')

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

query = '' # I always initialize varibles
query = '''select avg(price),view, date_built, sqft_living
            from parquet_home_sales
                where price >=350000
            GROUP BY price, view, date_built, sqft_living
 '''

start_time = time.time()

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+----------+-----------+
|avg(price)|view|date_built|sqft_living|
+----------+----+----------+-----------+
|  398750.0|  28|      2017|       2258|
|  406369.0|  12|      2012|       2439|
|  421355.0|   9|      2016|       1981|
|  360203.0|   1|      2014|       2273|
|  367581.0|  36|      2010|       2082|
|  377410.0|  40|      2015|       1790|
|  358842.0|  34|      2016|       1900|
|  356872.0|   4|      2016|       2092|
|  403431.0|  43|      2010|       2340|
|  413675.0|  28|      2016|       2064|
|  429295.0|  34|      2015|       1996|
|  797263.0|  72|      2012|       3634|
|  443712.0|   8|      2017|       2207|
|  440746.0|  48|      2013|       1783|
|  717361.0|  74|      2016|       4733|
|  412379.0|  22|      2015|       2430|
|  721804.0|  50|      2017|       4266|
|  446330.0|  35|      2017|       1845|
|  383624.0|   3|      2015|       1635|
|  369169.0|  41|      2011|       2345|
+----------+----+----------+-----------+
only showing top

In [19]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table parquet_home_sales")

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached
#exactly like class exercise
if spark.catalog.isCached("parquet_home_sales"):
  print("a table is still cached")
else:
  print("all clear")


all clear
